# Solving Sudoku Puzzle with Pyomo and Excel

This formulation is based on the chapter 5.4.2 from Pyomo documentation.

$$
\begin{align}
    \text{s.t.} & \sum_{r \in ROWS} y_{r,c,v} = 1 \quad & \forall c \in COLS, v \in VALUES \\
                & \sum_{c \in COLS} y_{r,c,v} = 1 \quad & \forall r \in ROWS, v \in VALUES \\
                & \sum_{v \in VALUES} y_{r,c,v} = 1 \quad & \forall r \in ROWS, c \in COLS \\
                & \sum_{r=1}^{3} \sum_{c=1}^{3} y_{(r + U),(c + W),v} & U, W \in \{0,3,6\} \\
                & y_{r,c,v} \in \{0,1\} \quad & \forall r \in ROWS, c \in COLS, v \in VALUES
\end{align}
$$

In [ ]:
import pandas as pd
import pyomo.environ as pyo

In [ ]:
# Read xlsx

board = pd.read_excel('./input/sudoku_input.xlsx', header=None)
board.index += 1
board.columns += 1
display(board)

### Create model

In [ ]:
model = pyo.ConcreteModel()

### Sets

In [ ]:
model.ROWS = pyo.Set(initialize=board.index)
model.COLS = pyo.Set(initialize=board.columns)
model.VALUES = pyo.Set(initialize=pyo.RangeSet(1, 9))

In [ ]:
# subsquares
model.U = pyo.Set(initialize=[0, 3, 6])
model.W = pyo.Set(initialize=[0, 3, 6])

### Variables

In [ ]:
model.y = pyo.Var(model.ROWS, model.COLS, model.VALUES, within=pyo.Binary)

### Constraints

In [ ]:
# the board numbers must be maintained

for r in model.ROWS:
    for c in model.COLS:
        if board.loc[r, c] in model.VALUES:
            model.y[r, c, board.loc[r, c]].fix(1)


In [ ]:
# exactly one number in each row

def row_cstr(model, r, v):
    return sum(model.y[r, :, v]) == 1

model.row_cstr = pyo.Constraint(model.ROWS, model.VALUES, rule=row_cstr)

In [ ]:
# exactly one number in each column

def col_cstr(model, c, v):
    return sum(model.y[:, c, v]) == 1

model.col_cstr = pyo.Constraint(model.COLS, model.VALUES, rule=col_cstr)

In [ ]:
# exactly one number in each subsquare

def subsquares_cstr(model, u, w, v):
    return sum(model.y[r+u, c+w, v] for r in range(1, 4) for c in range(1, 4)) == 1

model.subsquares_cstr = pyo.Constraint(model.U, model.W, model.VALUES, rule=subsquares_cstr)

In [ ]:
# exactly one number in each cell

def value_cstr(model, r, c):
    return sum(model.y[r, c, :]) == 1

model.value_cstr = pyo.Constraint(model.ROWS, model.COLS, rule=value_cstr)

### Objective

In [ ]:
model.obj = pyo.Objective(expr=1.0)

## Integer cut

The integer cut uses two sets. The first set $S_0$ consists of indices for those variables whose current solution is 0, and the second set $S_1$ consists of indices for those variables whose current solution is 1. Given the two sets, the integer cut constraint would prevent such a solution from appeating twice.

$$
\begin{align}
    \sum_{(r,c,v) \in S_0} y_{r,c,v} + \sum_{(r,c,v) \in S_1} (1 - y_{r,c,v}) \ge 1
\end{align}
$$

In [ ]:
# adding a new integer cut to the model

def add_integer_cut(model):
    # add the ConstraintList to store the IntegerCuts if it does not already exist
    if not hasattr(model, 'IntegerCuts'):
        model.IntegerCuts = pyo.ConstraintList()

    # add the integer cut corresponding to the current solution in the model
    cut_expr = 0.0
    for r in model.ROWS:
        for c in model.COLS:
            for v in model.VALUES:
                if not model.y[r, c, v].fixed:
                    if pyo.value(model.y[r, c, v]) >= 0.5:
                        cut_expr += (1.0 - model.y[r, c, v])
                    else:
                        cut_expr += model.y[r, c, v]
    model.IntegerCuts.add(cut_expr >= 1)

In [ ]:
# prints the solution stored in the model

def print_solution(model):
    for r in model.ROWS:
        print(' '.join(str(v) for c in model.COLS
                       for v in model.VALUES
                       if pyo.value(model.y[r, c, v]) >= 0.5))

In [ ]:
def solution_to_dataframe(model):
    grid = []

    for r in model.ROWS:
        row_values = [] * 9 # initialize an empty row
        for c in model.COLS:
            for v in model.VALUES:
                if pyo.value(model.y[r, c, v]) >= 0.5:
                    row_values.append(v)
        grid.append(row_values)
    
    df = pd.DataFrame(grid)
    return df

In [ ]:
solution_count = 0
dfs = {}

while 1:
    with pyo.SolverFactory("gurobi") as opt:
        results = opt.solve(model)
        if results.solver.termination_condition != pyo.TerminationCondition.optimal:
            print("All board solutions have been found")
            break

    solution_count += 1

    add_integer_cut(model)

    print("Solution #%d" % (solution_count))
    print_solution(model)
    
    df = solution_to_dataframe(model)

    # store the DataFrame in the dictionary with a key
    dfs[f'Solution_{solution_count}'] = df

In [ ]:
with pd.ExcelWriter('./sudoku_output.xlsx') as file:
    for key, dataframe in dfs.items():
        dataframe.to_excel(file, sheet_name=key)